The goal of this experiment is to see how well an explanation correlates with annotated bounding boxes.

In [ ]:
from torchvision import transforms as T
import torch
from minmax.lmm_models import vgg, resnet, densenet
from torchvision.models import vgg as vggn
import warnings
warnings.filterwarnings('ignore')

data_transforms = {
    'train': T.Compose([
        T.Resize((224,224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': T.Compose([
        T.Resize((224,224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'inverse': T.Compose([
        T.Normalize(mean=[0, 0, 0], std=[1/0.229, 1/0.224, 1/0.225]),
        T.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
    ]),
    'target': T.Compose([
        T.Resize((224,224)),
        T.ToTensor(),
        T.Lambda(lambda X: X < 0.007),
        T.Lambda(lambda X: X[0,:,:])
    ])
}

from torchvision.datasets import OxfordIIITPet
oxfordpets = OxfordIIITPet(
    root='~/datasets/',
    split='trainval',
    target_types='segmentation',
    transform=data_transforms['train'],
    target_transform=data_transforms['target']
)

testloader = torch.utils.data.DataLoader(
    oxfordpets,
    batch_size=1,
    num_workers=2,
    shuffle=True
)

netn = vggn.vgg19(pretrained=True)
net = vgg.vgg19(pretrained=True)

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
from minmax.mm_linearize import mmTensor, explainV1, explainV2, explainGradient, prediction_score
from sklearn import metrics
from pytorch_grad_cam import GradCAM
import numpy as np
import os
from datetime import datetime
from captum.attr import (
    Occlusion,
    NoiseTunnel,
    DeepLiftShap,
    Saliency,
    GuidedGradCam
)

OUTPUT_FOLDER = "Experiment_4"
try:
    os.mkdir(OUTPUT_FOLDER)
    print("Output Folder Created")
except FileExistsError:
    print("Output Folder Exists")


def auc(y, pred):
    y = y.flatten()
    pred = pred.flatten()
    fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
    return metrics.auc(fpr, tpr)

for i, (data, mask) in enumerate(testloader):
    R = []
    
    target = netn(data).argmax().item()
    
    # Guided GradCam
    method = {
        'name': 'Guided-GradCam',
        'time': datetime.now(),
        'cmap': plt.cm.Reds
    }
    timeGuidedGradCam = datetime.now()
    expl = GuidedGradCam(netn, netn.features[-1])
    explGuidedGradCam = expl.attribute(data, target=target)
    explGuidedGradCam = explGuidedGradCam.detach().numpy()
    explGuidedGradCam = explGuidedGradCam.sum(axis=0)
    explGuidedGradCam = np.linalg.norm(explGuidedGradCam, axis=0)
    method['expl'] = explGuidedGradCam
    method['score'] = auc(mask, method['expl'])
    method['time'] = (datetime.now() - method['time']).seconds
    R.append(method)

    # SmoothGrad
    method = {
        'name': 'SmoothGrad',
        'time': datetime.now(),
        'cmap': plt.cm.Reds
    }
    saliency = Saliency(netn)
    nt = NoiseTunnel(saliency)
    explSmoothGrad = nt.attribute(data, nt_type='smoothgrad', nt_samples=15, target=target)
    explSmoothGrad = explSmoothGrad.detach().numpy().sum(axis=0)
    explSmoothGrad = np.linalg.norm(explSmoothGrad, axis=0)
    method['expl'] = explSmoothGrad
    method['score'] = auc(mask, method['expl'])
    method['time'] = (datetime.now() - method['time']).seconds
    R.append(method)
    
    # DeepLiftShap
    method = {
        'name': 'DeepLiftShap',
        'time': datetime.now(),
        'cmap': plt.cm.Reds
    }
    dist = torch.cat([data * 0, data * 1])
    expl = DeepLiftShap(netn)
    explShapley = expl.attribute(data, baselines=dist, target=target)
    explShapley = explShapley.detach().numpy().sum(axis=0)
    explShapley = np.linalg.norm(explShapley, axis=0)
    method['expl'] = explShapley
    method['score'] = auc(mask, method['expl'])
    method['time'] = (datetime.now() - method['time']).seconds
    R.append(method)

    # Occlusion
    method = {
        'name': 'Occlusion',
        'time': datetime.now(),
        'cmap': plt.cm.Reds
    }
    expl = Occlusion(netn)
    explOcclusion = expl.attribute(data, target=target, strides=(3,25,25), sliding_window_shapes=(3,50,50))
    explOcclusion = explOcclusion.detach().numpy().sum(axis=0).sum(axis=0)
    if not target: explOcclusion = -explOcclusion
    explOcclusion -= explOcclusion.min()
    explOcclusion /= explOcclusion.max()
    method['expl'] = explOcclusion
    method['score'] = auc(mask, method['expl'])
    method['time'] = (datetime.now() - method['time']).seconds
    R.append(method)

    # Gradient
    method = {
        'name': 'Gradient',
        'time': datetime.now(),
        'cmap': plt.cm.Reds
    }
    explGradient, _ = explainGradient(net, data)
    method['expl'] = explGradient
    method['score'] = auc(mask, method['expl'])
    method['time'] = (datetime.now() - method['time']).seconds
    R.append(method)

    # RangeGrad
    method = {
        'name': 'RangeGrad',
        'time': datetime.now(),
        'cmap': plt.cm.Reds
    }
    explRangeGrad, _ = explainV2(net, data, scale=1/100, width=1/1000)
    method['expl'] = explRangeGrad
    method['score'] = auc(mask, method['expl'])
    method['time'] = (datetime.now() - method['time']).seconds
    R.append(method)

    # Mask 
    method = {
        'name': 'Mask',
        'expl': mask.sum(axis=0),
        'cmap': plt.cm.Reds
    }
    R.append(method)

    # Image
    method = {
        'name': 'Original',
        'expl': data_transforms['inverse'](data)[0].movedim(0,2).numpy(),
        'cmap': None
    }
    R.append(method)
    
    
    # Plot all versions of the explanation
    fig, axs = plt.subplots(2, 4)
    for ax in axs.ravel(): ax.axis('off')
    fig.set_size_inches(17,10)
    
    for j, d in enumerate(R):
        p = divmod(j, 4)
        axs[p].imshow(d['expl'], cmap=d['cmap'])
        if 'score' in d:
            axs[p].set_title("{name} {score:.3f}".format(**d))
        else:
            axs[p].set_title("{name}".format(**d))
        filename = "{dir}/{i}_{name}.png".format(i=i, dir=OUTPUT_FOLDER, **d)
        plt.imsave(filename, d['expl'], cmap=d['cmap'])
    plt.show()
    
    # Add to data table
    """
    with open("Experiment_4/scores_trained.txt", "a") as f:
        f.write("{},{},{},{},{},{}\n".format(scoreGuidedGradCam, scoreSmoothGrad, scoreShapley, scoreOcclusion, scoreGradient, scoreRangeGrad))
    with open("Experiment_4/times_trained.txt", "a") as f:
        f.write("{},{},{},{},{},{}\n".format(timeGuidedGradCam, timeSmoothGrad, timeShapley, timeOcclusion, timeGradient, timeRangeGrad))
    """
    if i > 10: break

In [ ]:
import csv
import statistics as stats
import scipy.stats as st
from scipy.stats import t
import numpy as np

# define the file path and read the file
filepath = 'Experiment_4/scores_trained.txt'
data = []
with open(filepath, 'r') as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        data.append(row)

# define the file path and read the file
filepath = 'Experiment_4/times_trained.txt'
time_data = []
with open(filepath, 'r') as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        time_data.append(row)

# transpose the data to make columns into rows
data = list(map(list, zip(*data)))
time_data = list(map(list, zip(*time_data)))
labels = ["Guided-GradCam", "SmoothGrad", "DeepLiftShapley", "Occlusion", "Gradient", "RangeGrad"]

print("\\begin{tabular}{r|rl|rl}")
print("\tMethod & \\multicolumn{2}{c}{AUC} & \\multicolumn{2}{c}{Runtime (s)} \\\\\\hline")
for i in range(6):
    # calculate mean and confidence interval
    d = [float(j) for j in data[i]]
    time_d = [float(j) for j in time_data[i]]
    mean = stats.mean(d)
    ci = t.interval(0.95, len(d)-1, scale=st.sem(d))
    time_mean = stats.mean(time_d)
    time_ci = t.interval(0.95, len(time_d)-1, scale=st.sem(time_d))
    print("\t{} & {:.3f} & \\textpm {:.3f} & {:.1f} & \\textpm {:.1f} \\\\".format(labels[i], mean, ci[1], time_mean, time_ci[1]))
print("\\end{tabular}")